In [24]:
import os
root_dir = "/home/jupyter/fun/kaggle/kaggle-competitions/aditya"
data_dir = "/home/jupyter/fun/data/my_dear_watson"

import pandas as pd
from tqdm import tqdm

## Data works

In [12]:
train = pd.read_csv(os.path.join(data_dir, "train.csv"))
test = pd.read_csv(os.path.join(data_dir, "test.csv"))
sample_submission = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))

## Available model testing

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

/opt/conda/envs/docvqa_python39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [15]:
model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

In [52]:
## kaggle_requirements - 0 for entailment, 1 for neutral, 2 for contradiction)
label_names = [0, 1, 2]
test_predictions = []
for idx, row in tqdm(test.iterrows()) :
    
    premise = test.loc[idx, "premise"]
    hypothesis = test.loc[idx, "hypothesis"]
    
    input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
    output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    
    prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
    test_predictions.append(list(sorted(prediction.items(), key=lambda x : x[1], reverse = True)[0]))

5195it [02:20, 36.98it/s]


In [53]:
prediction_df = pd.DataFrame(test_predictions, columns = ['inference', 'predict_proba'])

In [54]:
submission = pd.concat([sample_submission, prediction_df], axis = 1)

In [56]:
submission_file = submission[['id', 'inference']]
submission_file.columns = ['id', 'prediction']
submission_file.to_csv('mDeBERTa-v3-base-xnli-multilingual-nli-2mil7_baseline.csv', index = False)